# K-Prototypes Tech Talk
### By Palermo Penano

The summary below closely follows the original paper, [Huang 1998](https://link.springer.com/article/10.1023/A:1009769707641).

## Outline
* A review of the K-means clustering algorithm
* Categorical variables and why they shouldn't be used with K-means
* K-prototypes
    * K-modes
    * Dissimilarity measure
    * K-means + K-modes --> K-prototypes
* Demo: K-prototypes in Python

## References

* Python implementation
    * https://pypi.org/project/kmodes/
    * https://github.com/nicodv/kmodes
* Other articles and notebooks using k-prototypes
    * https://www.kaggle.com/rohanadagouda/unsupervised-learning-using-k-prototype-and-dbscan
    * https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb

* Source for this demo
    * Dataset
        * https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/
    * Analysis
        * https://towardsdatascience.com/clustering-datasets-having-both-numerical-and-categorical-variables-ed91cdca0677
        * https://www.kaggle.com/paulinan/bank-customer-segmentation

# Reviewing K-means

## K-means algorithm
* Initialize with a random set of cluster vectors (these are m-dimensional vectors where m is the number of features in the dataset)
* In a loop:
    * Allocate each point in the data to the nearest cluster based on the Euclidean distance (or other p-norm s)
    * For each cluster, update the cluster vector by calculating the mean of all the records assigned to that cluster (average of each component across all vectors belonging to the same cluster).
    * Repeat the first step by checking and reallocating each data point based on the new cluster vectors
    * Repeat until desired number of iteration is reached or record changes cluster
    
Here's a visualization of the algorithm
    
![kmeans_algo](./imgs/kmeans_algo.png)
[Source](https://en.wikipedia.org/wiki/K-means_clustering)

## Categorical variables and why they shouldn't be used with K-means

Categorical variables are data that takes one of a limited and usually fixed number of possible values. If they can be ordered, then it is called `ordinal`. If it cannot be ordered, then it is called `nominal`. Here are some examples:

    * ordinal: risk rating, rank, day of the week
    * nominal: political party, region of residence, occupation

There are several ways these variables are used in models. For unsupervised ML, some approaches include one-hot encoding, label encoding, feature aggregation, or excluding the feature from the model.

For one-hot encoding, the categorical is converted to several binary features each of which representing a given category in the original feature. This works when there are few categories but become a computational burden when categories exceeds hundreds or thousands of unique values (e.g. zip codes).

Label encoding recasts the original values of the categorical variable to a number. For example, if you had feature for pets the encoding may assign dogs to 1, cats to 2, birds to 3, etc. This is a popular technique for decision tree classifiers but inappropriate for K-means as there are no meaningful interpretation of distance between the numbers assigned to each class.

In the case of a categorical feature with high cardinality, one solution is to aggregate them. For example, zip codes can be aggregated to a region containing many zip codes. Following the aggregation, one can then apply one-hot encoding.

When the approaches above fails or is infeasible to apply, the data scientist may exclude categorical feature all together from the model.

For numerical data, it is natural to define the center of mass for a collection of points using common distance metrics, such as the Euclidean distance. Say you have three points in your dataset containing annual income: `[$100k, $45k, $65k]`. One metric for the center of mass is the average, which in this case is `$70k`. 

But instead of a numerical data like income, suppose you had data on hair color: `[blond, red, black]`. How can one define center of mass so that if you were to observe a new data point, say a person with purple hair, you can assign them to a cluster of hair colors similar to theirs? Most data scientists at this point would simply omit the categorical variables when undertaking a clustering study.

# K-prototypes

K-prototypes combines k-means and k-modes in a way that enables the use of both numeric and categorical variables. I explain how k-modes work first, then explain how it is used with k-means to form k-prototypes.

### K-modes

The k-modes algorithm differs from k-means in the following way:

- Instead of of a p-norm distance metric such as the Euclidean distance, use an alternative dissimilarity measure for categorical objects (this avoids having to do one-hot encoding if one were to use the Euclidean distance, for example).
- Instead of the average of a set of points to define a cluster centroid, use the mode where a frequency-based approach is used to find modes for the clusters

### Dissimilarity measure for categorical attributes

Consider the following example of three individuals and categorical data on their occupation, marital status, and hobbies.

```
x1 = [finance, married, golf] 
x2 = [finance, married, squash] 
x3 = [healthcare, single, soccer]
```

Intuitively, we can say that x1 and x2 are similar to one another. x3 is most dissimilar in the group.

One way we can capture dissimilarity is by the function described by following

<img src="./imgs/dissimilarity_measure.png" alt="drawing" width="250"/>

For two vector of categorical variables with `m` dimensions, the greater the value of the function `d`, the greater the dissimilarity between the two vectors.

To find the "center" of a set of categorical value vectors, k-modes uses the mode. The mode itself need not be part of the original set. In the example above with just 3 data points, you can have a mode that is `[finance, single, golf]`, which is not part of the original dataset. 

The paper describes an approach for finding the mode of a set of categorical valued vectors that I'm omitting from this tutorial because it is beyond my technical capability to understand it. Intuitively, the mode is a vector Q such that this Q minimizes the total dissimilarity between itself and the other vectors in the set:


<img src="./imgs/cat_mode.png" alt="drawing" width="500"/>

Now that we have an approach for defining how close two vectors contaning categorical values are and a technique for determining the center of mass for a set of categorical-valued vectors, we can use a similar algorithm as k-means to find the clusters for an entire dataset containing only categorical valued vectors:

#### The k-modes algorithm

* Select the k initial modes, one for each cluster
* In a loop, 
    * Allocate each point in the dataset to the nearest cluster based on the dissimilarity measure defined above
    * Update the mode of the cluster based on the set of points allocated to it
    * Repeat the first step by checking and reallocating each data point based on the new cluster modes
    * Terminate when no data point changes cluster or the desired number of iteration is reached

### Combining k-means and k-modes to form k-prototypes

Combining k-means and k-modes requires a function that combines both the p-norm distance metric and the dissimilarity measure function. Formally, the function is formulated as 

<img src="./imgs/combined_euc_dissm.png" alt="drawing" width="300"/>

Similar to the k-means and k-modes algorithm above, the optimization problems boils down to finding the cluster vectors (that now contain both numeric and categorical values) such that the sum of the p-norm distance metric and dissimilarity measure across all k cluster vectors is minimized.

To the mathematically inclined, the cost function that is minimized is 

<img src="./imgs/kproto_costfunc.png" alt="drawing" width="450"/>

where `k` is the number of clusters, `n` the number of data points, and `p` and `m` are the dimensions of the numerical and categorical features. The `gamma` hyperparameter controls how much weight is assigned to the categorical features relative to numerical features

# Demo: K-prototypes in Python

In [1]:
import warnings
from typing import List, Tuple

import numpy as np
import pandas as pd 
import random
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from scipy import stats
from matplotlib import style

from kmodes.kprototypes import KPrototypes
from kmodes.util import dissim               # Import module containing dissimilarity functions for categoricals
                                             # For list of all dissimilarity functions, see https://github.com/nicodv/kmodes/blob/master/kmodes/util/dissim.py

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.3f}'.format

# Clustering German Credit Data

In [2]:
col_names = [
    "chk_acct",          # 1
    "duration",          # 2
    "credit_his",        # 3
    "purpose",           # 4
    "amount",            # 5
    "saving_acct",       # 6
    "present_emp",       # 7
    "installment_rate",  # 8
    "sex",               # 9
    "other_debtor",      # 10          
    "present_resid",     # 11
    "property",          # 12
    "age",               # 13
    "other_install",     # 14
    "housing",           # 15
    "n_credits",         # 16
    "job",               # 17
    "n_people",          # 18
    "telephone",         # 19
    "foreign",           # 20
    "response"           # 21
]

df = pd.read_csv('./data/german/german.data',
                 sep=' ', 
                 header=None,
                 names=col_names)

In [3]:
# Restrict to this list of variables to simplify problem
# See https://towardsdatascience.com/clustering-datasets-having-both-numerical-and-categorical-variables-ed91cdca0677

cont_cols = [
    "duration",          # 2
    "amount",            # 5
    "installment_rate",  # 8
    "present_resid",     # 11
    "age",               # 13
    "n_credits",         # 16
    "n_people"           # 18
]
cat_cols = [
    "chk_acct",          # 1
    "credit_his",        # 3
    "present_emp",       # 7
    "sex",               # 9
    "property",          # 12
    "housing"            # 15
]

In [4]:
df = df.loc[:, cont_cols+cat_cols]

In [5]:
df.head().T

,0,1,2,3,4
duration,6,48,12,42,24
amount,1169,5951,2096,7882,4870
installment_rate,4,2,2,2,3
present_resid,4,2,3,4,4
age,67,22,49,45,53
n_credits,2,1,1,1,2
n_people,1,1,2,2,2
chk_acct,A11,A12,A14,A11,A11
credit_his,A34,A32,A34,A32,A33
present_emp,A75,A73,A74,A74,A73


In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
duration            1000 non-null int64
amount              1000 non-null int64
installment_rate    1000 non-null int64
present_resid       1000 non-null int64
age                 1000 non-null int64
n_credits           1000 non-null int64
n_people            1000 non-null int64
chk_acct            1000 non-null object
credit_his          1000 non-null object
present_emp         1000 non-null object
sex                 1000 non-null object
property            1000 non-null object
housing             1000 non-null object
dtypes: int64(7), object(6)
memory usage: 101.7+ KB


In [7]:
df[cont_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
duration,"1,000.000",20.903,12.059,4.000,12.000,18.000,24.000,72.000
amount,"1,000.000","3,271.258","2,822.737",250.000,"1,365.500","2,319.500","3,972.250","18,424.000"
installment_rate,"1,000.000",2.973,1.119,1.000,2.000,3.000,4.000,4.000
present_resid,"1,000.000",2.845,1.104,1.000,2.000,3.000,4.000,4.000
age,"1,000.000",35.546,11.375,19.000,27.000,33.000,42.000,75.000
n_credits,"1,000.000",1.407,0.578,1.000,1.000,1.000,2.000,4.000
n_people,"1,000.000",1.155,0.362,1.000,1.000,1.000,1.000,2.000


In [8]:
def summarize_cats(df: pd.DataFrame, cat_cols: List[str] = []) -> pd.DataFrame:
    '''Create table summarizing categorical variables
    To display more values in Values col, set
    pd.set_option('display.max_colwidth', 100)
    '''

    df_cats = df.loc[:, cat_cols]

    print(f"Dataset Shape: {df_cats.shape}")
    summary = pd.DataFrame(df_cats.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Column Name'] = summary['index']
    summary = summary[['Column Name', 'dtypes']]
    summary['Missing'] = df_cats.isnull().sum().values
    summary['Uniques'] = df_cats.nunique().values

    for name in summary['Column Name'].value_counts().index:

        # List unique values
        list_uniques = [str(v) for v in df_cats[name].unique()]
        summary.loc[summary['Column Name'] == name,
                    'Values'] = ' | '.join(list_uniques)

        # Calculate entropy
        shares = df_cats[name].value_counts(normalize=True)
        summary.loc[summary['Column Name'] == name, 'Entropy'] = round(
            stats.entropy(shares, base=2), 2)

    return summary

In [9]:
summarize_cats(df, cat_cols=cat_cols)

Dataset Shape: (1000, 6)


,Column Name,dtypes,Missing,Uniques,Values,Entropy
0,chk_acct,object,0,4,A11 | A12 | A14 | A13,1.800
1,credit_his,object,0,5,A34 | A32 | A33 | A30 | A31,1.710
2,present_emp,object,0,5,A75 | A73 | A74 | A71 | A72,2.160
3,sex,object,0,4,A93 | A92 | A91 | A94,1.530
4,property,object,0,4,A121 | A122 | A124 | A123,1.950
5,housing,object,0,3,A152 | A153 | A151,1.140


# Clustering using K-Prototypes

In [10]:
# Scale numerical features using Z-scores

display(df[cont_cols].describe().T)
scaler = StandardScaler()
df.loc[:, cont_cols] = scaler.fit_transform(df.loc[:, cont_cols])
display(df[cont_cols].describe().T)

,count,mean,std,min,25%,50%,75%,max
duration,"1,000.000",20.903,12.059,4.000,12.000,18.000,24.000,72.000
amount,"1,000.000","3,271.258","2,822.737",250.000,"1,365.500","2,319.500","3,972.250","18,424.000"
installment_rate,"1,000.000",2.973,1.119,1.000,2.000,3.000,4.000,4.000
present_resid,"1,000.000",2.845,1.104,1.000,2.000,3.000,4.000,4.000
age,"1,000.000",35.546,11.375,19.000,27.000,33.000,42.000,75.000
n_credits,"1,000.000",1.407,0.578,1.000,1.000,1.000,2.000,4.000
n_people,"1,000.000",1.155,0.362,1.000,1.000,1.000,1.000,2.000


,count,mean,std,min,25%,50%,75%,max
duration,"1,000.000",0.000,1.001,-1.402,-0.739,-0.241,0.257,4.239
amount,"1,000.000",0.000,1.001,-1.071,-0.675,-0.337,0.248,5.371
installment_rate,"1,000.000",0.000,1.001,-1.765,-0.870,0.024,0.918,0.918
present_resid,"1,000.000",-0.000,1.001,-1.672,-0.766,0.141,1.047,1.047
age,"1,000.000",0.000,1.001,-1.455,-0.752,-0.224,0.568,3.470
n_credits,"1,000.000",-0.000,1.001,-0.705,-0.705,-0.705,1.027,4.491
n_people,"1,000.000",-0.000,1.001,-0.428,-0.428,-0.428,-0.428,2.335


In [11]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
duration            1000 non-null float64
amount              1000 non-null float64
installment_rate    1000 non-null float64
present_resid       1000 non-null float64
age                 1000 non-null float64
n_credits           1000 non-null float64
n_people            1000 non-null float64
chk_acct            1000 non-null object
credit_his          1000 non-null object
present_emp         1000 non-null object
sex                 1000 non-null object
property            1000 non-null object
housing             1000 non-null object
dtypes: float64(7), object(6)
memory usage: 101.7+ KB


In [12]:
# Get index of categorical columns
cat_cols_idx = [df.columns.get_loc(c) for c in cat_cols if c in df]

In [19]:
# Fit and predict kprototypes

# For more details on parameters, see source code:
# https://github.com/nicodv/kmodes/blob/master/kmodes/kprototypes.py#L335

kp = KPrototypes(
        n_clusters=5,
        max_iter=100,                      # Number of iteration (cluster updates) per run (n_init)
        cat_dissim=dissim.matching_dissim, # Dissimilarity function to use for cat. features
        init='Cao',                        # Initialization method (see orignal paper for details)
        n_init=10,                         # Number of time the k-modes algorithm will be run with different centroid seeds.
        gamma=None,                        # Weighing factor that determines relative importance of cont. vs cat. features
        verbose=300,                       # Verbosity of output during training
        random_state=None,
        n_jobs=-1,
)

clusters = kp.fit_predict(df, categorical=cat_cols_idx)

Best run was number 3


In [14]:
kp.cluster_centroids_

array([['1.6631135941260118', '1.924092350533361', '-0.3004618409445169',
        '-0.047506370048549025', '-0.003068563391435645',
        '0.01353529763036709', '-0.12127195606936059', 'A12', 'A32',
        'A73', 'A93', 'A123', 'A152'],
       ['-0.25270985191094797', '-0.1715255782549233',
        '-0.14833106017745493', '0.08223086466253096',
        '0.2761455759566796', '0.26004816577989853', '2.334868926348068',
        'A14', 'A32', 'A75', 'A93', 'A121', 'A152'],
       ['-0.16572066311769731', '-0.4175891436268861',
        '0.7232546349927409', '-0.35953107266581125',
        '-0.5121723053649498', '-0.20933757280044665',
        '-0.42828956637154286', 'A14', 'A32', 'A73', 'A93', 'A123',
        'A152'],
       ['-0.3800822246797666', '-0.14047695640434216',
        '-1.2104650880652281', '-0.1513382708010449',
        '-0.5105232364042365', '-0.15575370983763165',
        '-0.4282895663715408', 'A14', 'A32', 'A73', 'A92', 'A123',
        'A152'],
       ['-0.30411488996258

In [15]:
# Output is an array with cluster assignment for each record in original dataset
display(clusters.shape)
display(clusters)

(1000,)

array([4, 0, 1, 0, 1, 0, 4, 0, 4, 2, 2, 0, 3, 4, 3, 2, 4, 0, 0, 1, 4, 1,
       1, 4, 3, 3, 4, 4, 2, 0, 2, 3, 3, 4, 2, 0, 0, 2, 3, 2, 2, 2, 1, 3,
       4, 4, 2, 3, 3, 2, 2, 3, 2, 3, 4, 1, 3, 0, 2, 0, 3, 4, 4, 0, 2, 1,
       3, 2, 2, 2, 0, 4, 1, 0, 4, 4, 2, 4, 0, 3, 3, 4, 2, 4, 4, 2, 2, 0,
       1, 2, 4, 4, 1, 2, 4, 0, 4, 2, 4, 1, 4, 2, 2, 2, 3, 0, 1, 3, 3, 3,
       1, 2, 2, 0, 2, 0, 0, 3, 3, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 0, 0,
       3, 4, 0, 2, 0, 4, 2, 3, 3, 2, 2, 3, 2, 0, 3, 4, 0, 4, 1, 3, 2, 0,
       0, 3, 1, 2, 3, 1, 1, 2, 2, 4, 2, 3, 2, 2, 3, 4, 1, 2, 2, 2, 3, 0,
       3, 4, 1, 4, 0, 0, 1, 4, 1, 2, 1, 4, 2, 3, 4, 1, 1, 3, 2, 3, 3, 4,
       2, 2, 4, 2, 2, 2, 1, 1, 4, 2, 3, 4, 1, 4, 4, 4, 4, 3, 2, 3, 3, 4,
       2, 2, 2, 1, 2, 2, 0, 4, 2, 2, 2, 4, 2, 2, 1, 1, 0, 4, 2, 1, 2, 1,
       1, 2, 1, 2, 4, 2, 3, 2, 4, 3, 2, 3, 3, 0, 2, 2, 3, 4, 3, 4, 4, 4,
       1, 1, 0, 2, 3, 2, 2, 4, 0, 0, 0, 1, 3, 4, 3, 2, 2, 2, 4, 4, 2, 0,
       1, 0, 4, 4, 3, 0, 1, 4, 0, 0, 3, 2, 4, 2, 4,

In [16]:
df['clusters'] = clusters

In [17]:
# Continuous variablesby clusters
df.groupby('clusters')[cont_cols].describe().T

clusters                     0       1       2       3       4
duration         count 135.000 140.000 339.000 205.000 181.000
                 mean    1.663  -0.253  -0.166  -0.380  -0.304
                 std     0.930   0.897   0.680   0.631   0.734
                 min    -1.236  -1.402  -1.236  -1.236  -1.319
                 25%     1.253  -0.905  -0.739  -0.739  -0.739
                 50%     1.501  -0.490  -0.241  -0.490  -0.490
                 75%     2.248   0.257   0.257   0.257   0.257
                 max     4.239   2.248   1.999   2.248   2.248
amount           count 135.000 140.000 339.000 205.000 181.000
                 mean    1.924  -0.172  -0.418  -0.140  -0.361
                 std     1.092   0.654   0.442   0.559   0.579
                 min    -0.078  -1.006  -1.062  -1.071  -1.040
                 25%     1.175  -0.678  -0.725  -0.544  -0.743
                 50%     1.669  -0.306  -0.515  -0.246  -0.542
                 75%     2.534   0.116  -0.202   0.134  -0.179
                 max     5.371   2.606   1.172   2.023   1.628
installment_rate count 135.000 140.000 339.000 205.000 181.000
                 mean   -0.300  -0.148   0.723  -1.210   0.355
                 std     0.963   1.040   0.370   0.470   0.811
                 min    -1.765  -1.765   0.024  -1.765  -1.765
                 25%    -0.870  -0.870   0.918  -1.765   0.024
                 50%    -0.870   0.024   0.918  -0.870   0.918
                 75%     0.918   0.918   0.918  -0.870   0.918
                 max     0.918   0.918   0.918   0.024   0.918
present_resid    count 135.000 140.000 339.000 205.000 181.000
                 mean   -0.048   0.082  -0.360  -0.151   0.817
                 std     0.991   0.968   0.951   1.029   0.507
                 min    -1.672  -1.672  -1.672  -1.672  -0.766
                 25%    -0.766  -0.766  -0.766  -0.766   1.047
                 50%     0.141   0.141  -0.766  -0.766   1.047
                 75%     1.047   1.047   0.141   1.047   1.047
                 max     1.047   1.047   1.047   1.047   1.047
age              count 135.000 140.000 339.000 205.000 181.000
                 mean   -0.003   0.276  -0.512  -0.511   1.326
                 std     0.906   0.765   0.548   0.636   0.923
                 min    -1.279  -1.367  -1.367  -1.455  -0.752
                 25%    -0.708  -0.224  -0.928  -1.015   0.568
                 50%    -0.224   0.216  -0.664  -0.664   1.271
                 75%     0.568   0.656  -0.136  -0.224   2.063
                 max     2.854   3.382   1.623   1.799   3.470
n_credits        count 135.000 140.000 339.000 205.000 181.000
                 mean    0.014   0.260  -0.209  -0.156   0.357
                 std     0.979   1.180   0.806   0.861   1.190
                 min    -0.705  -0.705  -0.705  -0.705  -0.705
                 25%    -0.705  -0.705  -0.705  -0.705  -0.705
                 50%    -0.705  -0.705  -0.705  -0.705   1.027
                 75%     1.027   1.027   1.027   1.027   1.027
                 max     4.491   4.491   2.759   2.759   4.491
n_people         count 135.000 140.000 339.000 205.000 181.000
                 mean   -0.121   2.335  -0.428  -0.428  -0.428
                 std     0.872   0.000   0.000   0.000   0.000
                 min    -0.428   2.335  -0.428  -0.428  -0.428
                 25%    -0.428   2.335  -0.428  -0.428  -0.428
                 50%    -0.428   2.335  -0.428  -0.428  -0.428
                 75%    -0.428   2.335  -0.428  -0.428  -0.428
                 max     2.335   2.335  -0.428  -0.428  -0.428

In [18]:
total = len(df)

for c in cat_cols:
    
    # Overall distribution of categories
    overall_dist = (
        pd.DataFrame(df[c].value_counts() / total).
        T.
        sort_index(axis=1).
        style.
        background_gradient(cmap='Blues', axis=1)
    )
    display(overall_dist)

    # Get counts across clusters and categorical column
    table = pd.pivot_table(
        df, values=[], index=["clusters"], columns=[c],
        aggfunc=len, margins=True, dropna=True, fill_value=0
    )

    # Divide by total across all cateogories for a given cluster
    table2 = table.div(table.iloc[:, -1], axis=0).iloc[:-1,:-1]

    # Apply background gradient to generate a heatmap
    display(table2.style.background_gradient(cmap='Blues', axis=1))
    print(30*"-")

,A11,A12,A13,A14
chk_acct,0.274,0.269,0.063,0.394


chk_acct,A11,A12,A13,A14
clusters,,,,
0,0.237037,0.407407,0.00740741,0.348148
1,0.342857,0.192857,0.0642857,0.4
2,0.289086,0.259587,0.0589971,0.39233
3,0.258537,0.302439,0.0829268,0.356098
4,0.237569,0.20442,0.0883978,0.469613


------------------------------


,A30,A31,A32,A33,A34
credit_his,0.04,0.049,0.53,0.088,0.293


credit_his,A30,A31,A32,A33,A34
clusters,,,,,
0,0.0814815,0.0592593,0.42963,0.17037,0.259259
1,0.0357143,0.0928571,0.442857,0.0928571,0.335714
2,0.0294985,0.0501475,0.634218,0.0678466,0.218289
3,0.0439024,0.0341463,0.619512,0.0829268,0.219512
4,0.0276243,0.0220994,0.375691,0.0662983,0.508287


...

property,A121,A122,A123,A124
clusters,,,,
0,0.103704,0.177778,0.385185,0.333333
1,0.335714,0.221429,0.242857,0.2
2,0.303835,0.244838,0.351032,0.100295
3,0.268293,0.253659,0.409756,0.0682927
4,0.348066,0.232044,0.237569,0.18232


------------------------------


,A151,A152,A153
housing,0.179,0.713,0.108


housing,A151,A152,A153
clusters,,,
0,0.148148,0.614815,0.237037
1,0.135714,0.7,0.164286
2,0.176991,0.775811,0.0471976
3,0.287805,0.687805,0.0243902
4,0.116022,0.707182,0.176796


------------------------------
